# Tutorial 1: CPTAC Data Introduction

The National Cancer Institute’s Clinical Proteomic Tumor Analysis Consortium (CPTAC) is a national effort to accelerate the understanding of the molecular basis of cancer through the application of large-scale proteome and genome analysis, or proteogenomics. CPTAC generates comprehensive proteomics and genomics data from clinical cohorts, typically with ~100 samples per tumor type. The graphic below summarizes the structure of each CPTAC dataset. For more information, visit the [NIH website](https://proteomics.cancer.gov/programs/cptac). 

<img src="img/Graphical_Abstract.png" alt="CPTAC cohort" width="700"/>

This Python package makes accessing CPTAC data easy with Python code and Jupyter notebooks. The package contains several tutorials which demonstrate data access and usage. This first tutorial serves as an introduction to the data to help users become familiar with what is included and how it is presented.

## Data Overview

Our package provides data access in a Python programming environment. If you have not installed Python or have not installed the package, see our installation documentation [here](https://paynelab.github.io/cptac/#installation).

Once we have the package installed and we're in our Python environment, we begin by importing the package with a standard Python import statement:

In [1]:
import cptac

cptac data is broken down into datasets by cancer type. To view the available datasets, call the `cptac.get_cancer_options()` function:

In [2]:
cptac.get_cancer_options()

['brca', 'ccrcc', 'coad', 'gbm', 'hnscc', 'lscc', 'luad', 'ov', 'pdac', 'ucec']

## Data Availability
The goals of CPTAC as a consortium include the broad and open dissemination of cancer proteogenomic data. The timing of a dataset's public release generally follows three stages: internal release to CPTAC investigators, public release with a publication embargo, and full public release. Each of the cancer types may be at a different data availability stage, depending on the date of data creation. In the Python `cptac` package, these three stages are dealt with as follows:

**Internally released data** requires a password to download.

**Embargoed release data** is publicly available, but prints an embargo statement every time you interact with the data.

**Public data** is fully released without restrictions.

## Exploring the data

`cptac` allows you to load the dataset into a Python variable, and you can use that variable to access and work with the data. To load a particular dataset into a variable, type the name you want to give the variable, followed by `=`, and then type `cptac.` and the name of the dataset in [UpperCamelCase](https://en.wikipedia.org/wiki/Camel_case) followed by two parentheses, e.g. `cptac.Ucec()` or `cptac.Ccrcc()`:

In [3]:
en = cptac.Ucec()

To see what data is available, use the `en.list_data_sources()` function. This displays the different types of data included in the dataset for this particular cancer type, each stored in a [pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [4]:
en.list_data_sources()

,Data type,Available sources
0,CNV,washu
1,circular_RNA,bcm
2,clinical,mssm
3,deconvolution_cibersort,washu
4,deconvolution_xcell,washu
5,miRNA,washu
6,phosphoproteomics,umich
7,proteomics,umich
8,somatic_mutation,"harmonized, washu"
9,transcriptomics,"bcm, broad, washu"


# Molecular Omics

You'll notice that some datatypes have more than one source, usually named after the organization which generated the data. Since not all of this data has been publicly released yet, we will use 'awg' data in these tutorials. AWG stands for "All Working Groups" and was generated collaboratively by many different groups, albeit with some inconsistencies between datasets.

Data can be accessed through the `get_dataframe` function, or through one of several helper "get" functions. For example, we can look at the proteomics data by using `en.get_proteomics()`. This returns a [pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe) containing the proteomic data. Each column in the proteomics dataframe is the quantitiative measurement for a particular protein. Each row in the proteomics dataframe is a sample of either a tumor or non-tumor from a cancer patient.

In [5]:
# These two methods of getting proteomics data are functionally equivalent:
# get_dataframe needs at least two arguments: datatype, and source
proteomics = en.get_dataframe('proteomics', 'umich')
# there is also a get function for each datatype, this is the same as get_dataframe but does not need the datatype argument
proteomics = en.get_proteomics('umich')

samples = proteomics.index
proteins = proteomics.columns
print("Samples:",samples[0:20].tolist()) #the first twenty samples
print("Proteins:",proteins[0:20].tolist()) #the first twenty proteins

Samples: ['C3L-00006', 'C3L-00008', 'C3L-00032', 'C3L-00084', 'C3L-00090', 'C3L-00098', 'C3L-00136', 'C3L-00137', 'C3L-00139', 'C3L-00143', 'C3L-00145', 'C3L-00156', 'C3L-00157', 'C3L-00161', 'C3L-00356', 'C3L-00358', 'C3L-00361', 'C3L-00362', 'C3L-00413', 'C3L-00449']
Proteins: [('ARF5', 'ENSP00000000233.5'), ('M6PR', 'ENSP00000000412.3'), ('ESRRA', 'ENSP00000000442.6'), ('FKBP4', 'ENSP00000001008.4'), ('NDUFAF7', 'ENSP00000002125.4'), ('FUCA2', 'ENSP00000002165.5'), ('DBNDD1', 'ENSP00000002501.6'), ('SEMA3F', 'ENSP00000002829.3'), ('CFTR', 'ENSP00000003084.6'), ('CYP51A1', 'ENSP00000003100.8'), ('USP28', 'ENSP00000003302.4'), ('SLC7A2', 'ENSP00000004531.10'), ('HSPB6', 'ENSP00000004982.3'), ('PDK4', 'ENSP00000005178.5'), ('USH1C', 'ENSP00000005226.7'), ('RALA', 'ENSP00000005257.2'), ('BAIAP2L1', 'ENSP00000005260.8'), ('TMEM132A', 'ENSP00000005286.4'), ('DVL2', 'ENSP00000005340.4'), ('RPAP3', 'ENSP00000005386.3')]


## Dataframe values

Values in the dataframe are protein abundance values. Values that read "NaN" mean that particular sample from that patient had no data for that particular protein. For the awg endometrial CPTAC proteomics data, a TMT-reference channel strategy was used. A detailed description of this strategy can be found at [Nature Protocols](https://www.nature.com/articles/s41596-018-0006-9) and also at [PubMed Central](https://www.ncbi.nlm.nih.gov/pubmed/?term=29988108). This strategy ratios each sample's abundance to a pooled reference. The ratio is then log transformed. Therefore positive values indicate a measurement higher than the pooled reference; negative values are lower than the pooled reference.

In [6]:
proteomics.head()

Name,ARF5,M6PR,ESRRA,FKBP4,NDUFAF7,FUCA2,DBNDD1,SEMA3F,CFTR,CYP51A1,...,SCRIB,WIZ,BPIFB4,LDB1,WIZ,TSGA10,RFX7,SWSAP1,MSANTD2,SVIL
Database_ID,ENSP00000000233.5,ENSP00000000412.3,ENSP00000000442.6,ENSP00000001008.4,ENSP00000002125.4,ENSP00000002165.5,ENSP00000002501.6,ENSP00000002829.3,ENSP00000003084.6,ENSP00000003100.8,...,ENSP00000501177.1,ENSP00000501256.3,ENSP00000501266.1,ENSP00000501277.1,ENSP00000501300.1,ENSP00000501312.1,ENSP00000501317.1,ENSP00000501355.1,ENSP00000501466.1,ENSP00000501521.1
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,-0.056513,0.016557,0.002569,0.389819,0.603610,-0.332543,-0.790426,NaN,0.822732,0.039134,...,0.161720,-0.884807,NaN,0.268247,0.125392,-0.880833,0.108554,0.107413,-0.085833,NaN
C3L-00008,0.549959,-0.206129,0.905784,-0.303631,0.018767,0.503513,0.950955,0.080142,NaN,-0.063213,...,NaN,0.054284,NaN,-0.106450,0.380557,-0.756099,0.264611,0.044423,-0.248319,-1.206596
C3L-00032,0.088681,-0.154447,-0.190515,0.170753,0.196356,0.544194,-0.179078,NaN,NaN,0.377405,...,-1.086905,0.055991,NaN,-0.021986,-0.229645,1.923986,NaN,-0.176694,-0.332384,-1.330653
C3L-00084,-0.846555,0.027740,NaN,0.178700,0.264054,-0.183548,0.077215,-0.247164,0.152277,-0.279549,...,-0.125796,0.944212,NaN,0.917409,0.026862,-0.885976,-0.006510,-0.014162,0.365158,NaN
C3L-00090,0.539019,0.956619,-0.039516,0.323656,0.064605,0.173433,-0.524325,-0.038590,-0.311486,0.309905,...,0.853362,-0.716947,NaN,-0.286277,-0.046076,0.089645,-0.444506,-0.072531,-0.463495,NaN


As seen in `en.list_data_sources()`, other omics data are also available (e.g. transcriptomics, copy number variation, phoshoproteomics).

The transcriptomics looks almost identical to the proteomics data, available in a pandas dataframe with the same convention. Each set of samples is consistent, meaning samples found in the endometrial proteomics data will be the same samples in all other endometrial dataframes.

In [7]:
transcriptomics = en.get_transcriptomics('bcm')
transcriptomics.head()

Name,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A2MP1,A3GALT2,...,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZYXP1,ZZEF1,hsa-mir-1253,hsa-mir-423
Database_ID,ENSG00000121410.12,ENSG00000268895.6,ENSG00000148584.15,ENSG00000175899.15,ENSG00000245105.4,ENSG00000166535.20,ENSG00000256661.1,ENSG00000256904.1,ENSG00000256069.7,ENSG00000184389.9,...,ENSG00000198455.4,ENSG00000070476.15,ENSG00000203995.10,ENSG00000232242.2,ENSG00000162378.13,ENSG00000159840.16,ENSG00000274572.1,ENSG00000074755.15,ENSG00000272920.1,ENSG00000266919.3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,2.54,5.11,3.60,13.75,6.45,7.08,1.80,0.00,2.60,1.16,...,10.17,10.61,5.54,0.0,11.85,10.60,0.0,11.87,0.0,0.0
C3L-00008,4.40,4.63,5.49,13.89,6.61,6.97,0.00,2.74,3.25,0.00,...,9.79,10.48,7.79,0.0,12.28,11.28,0.0,11.93,0.0,0.0
C3L-00032,4.83,7.26,3.73,14.48,6.91,9.56,0.98,0.00,3.26,0.00,...,9.43,9.97,6.48,0.0,11.72,10.37,0.0,11.70,0.0,0.0
C3L-00084,4.73,6.01,5.37,15.17,7.93,3.86,0.00,0.00,3.73,1.15,...,9.23,10.37,7.47,0.0,11.86,10.13,0.0,11.19,0.0,0.0
C3L-00090,4.14,6.24,5.69,13.87,6.79,4.32,0.00,0.00,3.23,0.00,...,9.69,9.64,7.60,0.0,11.98,10.31,0.0,11.45,0.0,0.0


# Clinical Data

The clinical dataframe lists clinical information for the patient associated with each sample (e.g. age, race, diabetes status, tumor size). 

In [8]:
clinical = en.get_clinical()
clinical.head()

cptac warning: Using source mssm for clinical data as no other sources provide this data. To remove this warning, pass mssm as the source parameter. (/var/folders/n2/95tlbmw5173fm8yjh4ffxggr0000gn/T/ipykernel_82825/241189121.py, line 1)


Name,tumor_code,discovery_study,discovery_study/type_of_analyzed_samples,consent/age,consent/sex,consent/race,consent/ethnicity,consent/ethnicity_race_ancestry_identified,consent/collection_in_us,consent/participant_country,...,follow-up/additional_treatment_for_new_tumor_radiation,follow-up/additional_treatment_for_new_tumor_pharmaceutical,follow-up/additional_treatment_for_new_tumor_immunological,follow-up/days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor,follow-up/cause_of_death,follow-up/days_from_date_of_initial_pathologic_diagnosis_to_date_of_death,Recurrence-free survival,Overall survial,"Recurrence status (1, yes; 0, no)","Survial status (1, dead; 0, alive)"
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,UCEC,Yes,Tumor_and_Normal,64,Female,White,Not-Hispanic or Latino,White,Yes,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737.0,0,0.0
C3L-00008,UCEC,Yes,Tumor,58,Female,White,Not-Hispanic or Latino,White,Yes,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,898.0,0,0.0
C3L-00032,UCEC,Yes,Tumor,50,Female,White,Not-Hispanic or Latino,White,Yes,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1275.0,0,0.0
C3L-00084,UCEC,Yes,Tumor,74,Female,White,Not-Hispanic or Latino,White,Yes,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,335.0,0,0.0
C3L-00090,UCEC,Yes,Tumor,75,Female,White,Not-Hispanic or Latino,White,Yes,United States,...,Yes,Yes,No,NaN,Other reasons,1281.0,50.0,1281.0,1,1.0


In addition to donating a tumor sample, some patients also had a normal sample taken for control and comparison. We can identify these samples by looking for samples marked "Tumor_and_Normal" in the "type_of_analyzed_samples" column. For example, patient C3L-00006 provided both a tumor sample and a normal sample. Since the data in these cases is for both tumor and normal samples, where a column would have identical values for both samples from the same patient (e.g., patient age and gender), the information is recorded only for the tumor sample.

In [9]:
clinical.loc[["C3L-00006","C3L-00361","C3L-01246"]]

Name,tumor_code,discovery_study,discovery_study/type_of_analyzed_samples,consent/age,consent/sex,consent/race,consent/ethnicity,consent/ethnicity_race_ancestry_identified,consent/collection_in_us,consent/participant_country,...,follow-up/additional_treatment_for_new_tumor_radiation,follow-up/additional_treatment_for_new_tumor_pharmaceutical,follow-up/additional_treatment_for_new_tumor_immunological,follow-up/days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor,follow-up/cause_of_death,follow-up/days_from_date_of_initial_pathologic_diagnosis_to_date_of_death,Recurrence-free survival,Overall survial,"Recurrence status (1, yes; 0, no)","Survial status (1, dead; 0, alive)"
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,UCEC,Yes,Tumor_and_Normal,64,Female,White,Not-Hispanic or Latino,White,Yes,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737.0,0,0.0
C3L-00361,UCEC,Yes,Tumor_and_Normal,64,Female,White,Not-Hispanic or Latino,WHITE,Yes,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1436.0,0,0.0
C3L-01246,UCEC,Yes,Tumor_and_Normal,62,Female,White,Not reported,Medical record does not state.,Yes,Other: Medical record does not state.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1132.0,0,0.0


# Mutation data

Each cancer dataset contains mutation data for the cohort. The data consists of all somatic mutations found for each sample (meaning there will be many lines for each sample). Each row lists the specific gene that was mutated, the type of mutation, and the location of the mutation. This data is a direct import of a MAF file.

In [10]:
somatic_mutations = en.get_somatic_mutation('harmonized')
somatic_mutations.head()

cptac warning: Your version of cptac (1.5.0rc1) is out-of-date. Latest is 1.1.2. Please run 'pip install --upgrade cptac' to update it. (/Users/blakemcgee/opt/anaconda3/lib/python3.9/threading.py, line 917)


Name,Gene,Mutation,Location,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Type,...,HGNC_UniProt_ID(supplied_by_UniProt),HGNC_Ensembl_ID(supplied_by_Ensembl),HGNC_UCSC_ID(supplied_by_UCSC),Oreganno_Build,Simple_Uniprot_alt_uniprot_accessions,dbSNP_TOPMED,HGNC_Entrez_Gene_ID(supplied_by_NCBI),COHORT,getz,washu
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,CPEB3,Missense_Mutation,p.R595C,22849.0,hg38,chr10,92081406,92081406,+,SNP,...,Q8NE35,ENSG00000107864,uc001khw.3,hg38,Q5T389|Q9NQJ7|Q9Y2E9,NaN,22849.0,UCEC,True,NaN
C3L-00006,SHANK2,Missense_Mutation,p.S1692R,22941.0,hg38,chr11,70473343,70473343,+,SNP,...,Q9UPX8,ENSG00000162105,uc058etp.1,NaN,C0SPG8|C0SPG9|Q3Y8G9|Q52LK2|Q9UKP1,NaN,22941.0,UCEC,True,True
C3L-00006,CTTN,Missense_Mutation,p.R119C,2017.0,hg38,chr11,70414605,70414605,+,SNP,...,Q14247,ENSG00000085733,uc001opw.5,NaN,Q8N707|Q96H99,NaN,2017.0,UCEC,True,NaN
C3L-00006,TPCN2,Missense_Mutation,p.V572I,219931.0,hg38,chr11,69083969,69083969,+,SNP,...,Q8NHX9,ENSG00000162341,uc001oos.3,hg38,Q9NT82,NaN,219931.0,UCEC,True,True
C3L-00006,PITPNM1,Silent,p.R478R,9600.0,hg38,chr11,67498646,67498646,+,SNP,...,O00562,ENSG00000110697,uc001oly.4,hg38,A6NME4|Q6T7X3|Q8TBN3|Q9BZ73,NaN,9600.0,UCEC,True,True


# Exporting dataframes

If you wish to export a dataframe to a file, simply call the dataframe's `to_csv` method, passing the path you wish to save the file to, and the value separator you want:

In [11]:
clinical = en.get_clinical('mssm')
clinical.to_csv(path_or_buf="clinical_dataframe.tsv", sep='\t')

## Downloading data

The cptac package stores the data files for each dataset on a remote server. When you first install cptac, you will have no data files. Data files will be automatically downloaded the first time you try to use them. If you won't have internet access and need to download the files beforehand, that can be done with the `cptac.download` function:

In [14]:
cptac.download({"umich": ["proteomics"]})

True

# Getting help with a dataset or function

To view the documentation for a dataset, pass it to the Python `help` function, e.g. `help(en)`. You can also view the documentation for just a specific function: `help(en.join_omics_to_omics)`.

In [13]:
help(en.join_omics_to_omics)

Help on method join_omics_to_omics in module cptac.cancers.cancer:

join_omics_to_omics(df1_name: str, df2_name: str, df1_source: str = None, df2_source: str = None, genes1: str = None, genes2: str = None, how: str = 'outer', quiet: bool = False, tissue_type: str = 'both') -> pandas.core.frame.DataFrame method of cptac.cancers.ucec.Ucec instance
    Take specified column(s) from one omics dataframe, and join to specified columns(s) from another omics dataframe. Intersection (inner join) of indices is used.
    
    Parameters:
    df1_name (str): Name of first omics dataframe to select columns from.
    df2_name (str): Name of second omics dataframe to select columns from.
    df1_source (str, optional): Name of source for the first omics dataframe.
    df2_source (str, optional): Name of source for the second omics dataframe.
    genes1 (str, or list or array-like of str, optional): Gene(s) for column(s) to select from df1_name. str if one key, list or array-like of str if multiple.
 